# Very simple Latent Dirichlet Allocation

LDA assumes that the documents in a corpus have been generated by a generative model, and tries to work backwards from the documents to find a set of topics that are likely to have generated the corpus.  
Documents in the corpus are bags-of-words.  


Code adapted from r code found here: http://brooksandrew.github.io/simpleblog/articles/latent-dirichlet-allocation-under-the-hood/

## Preparatory stuff

Import numpy to handle the matrices:

In [95]:
import numpy as np 

Generate a toy corpus. Each document is a string:

In [96]:
docs = [  'europe brussels brexit britain great again ukip',
          'brussels belgium leave europe',
          'europe eu britain brexit transport',
          'farage eu ukip brussels',
          'europe brexit great',
          'transport infrastructure cars pollution',
          'cars bikes cars',
          'cars pollution transport minister bikes',
          'bikes infrastructure transport europe',
          'pollution cars minister']
docs = [doc.split(' ') for doc in docs]

Set some parameters:

In [97]:
K = 2 # number of topics
alpha = 1 # hyperparameter. single value indicates symmetric dirichlet prior. higher=>scatters document clusters
beta = 0.001 # hyperparameter
iterations = 100 # iterations for collapsed gibbs sampling.  This should be a lot higher than 3 in practice.

Assign a word ID to each unique word:

In [98]:
wordIDs = {}
currentID = 0
for doc in docs:
	for i in range(len(doc)):
		if doc[i] not in wordIDs:
			wordIDs[doc[i]] = currentID
			currentID +=1
vocab = list(range(len(wordIDs)))

## Random initialization

Randomly assign topics to words in each document:

In [99]:
Cwt = np.zeros((K, len(vocab))) # initialize word-topic count matrix. wt refers to dimensions W * T
ta = [np.zeros((len(doc))) for doc in docs] # initialize topic assignment list

In [100]:
for d in range(len(docs)): # for each document
	for w in range(len(docs[d])): # for each token in document d 
		ta[d][w] = np.random.randint(K)
		ti = int(ta[d][w]) # topic index
		wi = wordIDs[docs[d][w]] # wordID for token w
		Cwt[ti,wi] +=1 # update word-topic matrix

Generate word-topic count matrix.

In [101]:
Cdt = np.zeros((len(docs), K)) # Document-topic matrix. dt refers to dimensions D * T
for d in range(len(docs)): # for each document d
	for t in range(K): # for each topic K
		for thing in ta[d]:
			if t == thing:
				Cdt[d,t] += 1 

With this random assignment, we already have (poor, random) <b>topic representations</b> of both all the <b>documents</b> and <b>word distributions of all the topics</b>.

## Learning

Collapsed Gibbs sampling happens here:

In [102]:
for i in range(iterations): # for each iteration
	for d in range(len(docs)): # for each document
		for w in range(len(docs[d])): # for each token
			t0 = int(ta[d][w]) # initial topic assignment to token w
			wid = wordIDs[docs[d][w]] # wordID of token w

			Cdt[d][t0] = Cdt[d][t0]-1 # don't include token w in our document-topic count matrix when sampling for token w
			Cwt[t0,wid] = Cwt[t0,wid]-1 # don't include token w in our word-topic count matrix when sampling for token w

            ## UPDATE TOPIC ASSIGNMENT FOR EACH WORD -- COLLAPSED GIBBS SAMPLING.
			denominator_a = sum(Cdt[d]) + K * alpha # number of tokens in document + number topics * alpha
			denominator_b = np.sum(Cwt, axis=1) + len(vocab) * beta # number of tokens in each topic + # of words in vocab * beta

			p_z = (Cwt[:,wid] + beta) / denominator_b * (Cdt[d,:] + alpha) / denominator_a # calculating probability word belongs to each topic
			t1 = np.random.choice(range(K), size=1, replace=True, p=p_z/sum(p_z)) # draw topic for word n from multinomial using probabilities calculated above


			ta[d][w] = t1 # update topic assignment list with newly sampled topic for token w.
			Cdt[d,t1] = Cdt[d,t1]+1 # re-increment document-topic matrix with new topic assignment for token w.
			Cwt[t1,wid] = Cwt[t1,wid]+1 # re-increment word-topic matrix with new topic assignment for token w.

The equation for the probabilty:

$p(z_i = j\ |\ \textbf{z}_{-i}, w_i, d_i \*) = \propto $  

## Results

Print out matrices:

In [103]:
print(Cwt, '\n\n', ta, '\n\n', Cdt)

[[ 5.  0.  0.  0.  0.  1.  2.  0.  0.  2.  4.  0.  2.  5.  3.  3.  0.]
 [ 0.  3.  3.  2.  2.  0.  0.  1.  1.  0.  0.  1.  0.  0.  0.  0.  2.]] 

 [array([ 0.,  1.,  1.,  1.,  1.,  0.,  0.]), array([ 1.,  1.,  1.,  0.]), array([ 0.,  0.,  1.,  1.,  0.]), array([ 1.,  0.,  0.,  1.]), array([ 0.,  1.,  1.]), array([ 0.,  0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.,  1.,  0.]), array([ 0.,  0.,  0.,  0.]), array([ 0.,  0.,  1.])] 

 [[ 3.  4.]
 [ 1.  3.]
 [ 3.  2.]
 [ 2.  2.]
 [ 1.  2.]
 [ 4.  0.]
 [ 3.  0.]
 [ 4.  1.]
 [ 4.  0.]
 [ 2.  1.]]


Analyse generated topics.  
Parameter <code>theta</code> normalizes the counts in the document-topic count matrix <code>Cdt</code> to compute the probability that a document belongs to each topic.  
<code>theta</code> is the <b>posterior mean</b> of the parameter <code>θ</code>.

In [57]:
theta = (Cdt + alpha) / np.reshape(np.sum(Cdt + alpha, axis=1),(len(docs), 1)) # topic probabilities per document
print(theta)

[[ 0.33333333  0.66666667]
 [ 0.5         0.5       ]
 [ 0.28571429  0.71428571]
 [ 0.66666667  0.33333333]
 [ 0.4         0.6       ]
 [ 0.5         0.5       ]
 [ 0.4         0.6       ]
 [ 0.57142857  0.42857143]
 [ 0.5         0.5       ]
 [ 0.6         0.4       ]]


<code>phi</code> normalizes the word-topic count matrix <code>Cwt</code> to compute the probabilities of words given topics.  
<code>phi</code> is  the <b>posterior mean</b> of the parameter <code>φ</code>.

In [52]:
phi = (Cwt + beta) / (np.sum(Cwt+beta)) # topic probabilities per word
#colnames(phi) = vocab
print(phi)

[[  2.37902650e-05   7.13945853e-02   2.37902650e-05   2.37902650e-05
    4.76043203e-02   2.38140553e-02   2.37902650e-05   2.38140553e-02
    2.37902650e-05   4.76043203e-02   9.51848504e-02   2.37902650e-05
    4.76043203e-02   2.37902650e-05   7.13945853e-02   7.13945853e-02
    2.37902650e-05]
 [  1.18975115e-01   2.37902650e-05   7.13945853e-02   4.76043203e-02
    2.37902650e-05   2.37902650e-05   4.76043203e-02   2.37902650e-05
    2.38140553e-02   2.37902650e-05   2.37902650e-05   2.38140553e-02
    2.37902650e-05   1.18975115e-01   2.37902650e-05   2.37902650e-05
    4.76043203e-02]]
